# Time series components

## Data

In [1]:
import pandas as pd

df = pd.read_csv('../data/airline-passengers.csv', parse_dates=['Month'], index_col='Month')
df = df.asfreq('MS')
df

,Passengers
Month,
1949-01-01,112
1949-02-01,118
...,...
1960-11-01,390
1960-12-01,432


## Individual component behaviour based on model

Components:

- Trend (T)
- Seasonality (S)
- Residual or Irregular Component (I)

Models:

- Additive model: $y_t = T_t + S_t + e_t$
- Multiplicative model: $y_t = T_t \times S_t \times e_t$

### Additive model

In [2]:
import statsmodels.api as sm

In [3]:
data = df['Passengers'].values
result = sm.tsa.seasonal_decompose(data, model='additive', period=12)

r = (df
 .assign(
    trend = result.trend,
    seasonal = result.seasonal,
    residual = result.resid)
 .dropna())

r

,Passengers,trend,seasonal,residual
Month,,,,
1949-07-01,148,126.791667,63.830808,-42.622475
1949-08-01,148,127.250000,62.823232,-42.073232
...,...,...,...,...
1960-05-01,472,472.750000,-4.506313,3.756313
1960-06-01,535,475.041667,35.402778,24.555556


In [4]:
r['model_result'] = r.trend + r.seasonal + r.residual
r

,Passengers,trend,seasonal,residual,model_result
Month,,,,,
1949-07-01,148,126.791667,63.830808,-42.622475,148.0
1949-08-01,148,127.250000,62.823232,-42.073232,148.0
...,...,...,...,...,...
1960-05-01,472,472.750000,-4.506313,3.756313,472.0
1960-06-01,535,475.041667,35.402778,24.555556,535.0


In [5]:
dfs = {}
dfs['additive'] = r

### Multiplicative model

In [6]:
r = df['Passengers'].values
result = sm.tsa.seasonal_decompose(data, model='multiplicative', period=12)

r = (df
 .assign(
    trend = result.trend,
    seasonal = result.seasonal,
    residual = result.resid)
 .dropna())

r

,Passengers,trend,seasonal,residual
Month,,,,
1949-07-01,148,126.791667,1.226556,0.951664
1949-08-01,148,127.250000,1.219911,0.953401
...,...,...,...,...
1960-05-01,472,472.750000,0.981378,1.017359
1960-06-01,535,475.041667,1.112776,1.012079


In [7]:
r['model_result'] = r.trend * r.seasonal * r.residual
r

,Passengers,trend,seasonal,residual,model_result
Month,,,,,
1949-07-01,148,126.791667,1.226556,0.951664,148.0
1949-08-01,148,127.250000,1.219911,0.953401,148.0
...,...,...,...,...,...
1960-05-01,472,472.750000,0.981378,1.017359,472.0
1960-06-01,535,475.041667,1.112776,1.012079,535.0


In [8]:
dfs['multiplicative'] = r

## Model comparison

### Numerical

In [ ]:
df = pd.concat(dfs, axis=1).melt(ignore_index=False).reset_index()
df.columns = ["month", "model", "component", "value"]

In [10]:
df

,month,model,component,value
0,1949-07-01,additive,Passengers,148.0
1,1949-08-01,additive,Passengers,148.0
...,...,...,...,...
1318,1960-05-01,multiplicative,model_result,472.0
1319,1960-06-01,multiplicative,model_result,535.0


## Visual comparison

In [10]:

from modules import utils
utils.configure_plotly_template(showlegend=True)

In [12]:
import plotly.express as px

fig = px.area(
    data_frame=df,
    x='month',
    y='value',
    color='component',
    facet_col='model',
    facet_row='component',
    width=1500,
    height=1000,
    facet_col_spacing=0.1,
)

fig.update_yaxes(matches=None)

for attr in dir(fig.layout):
    if attr.startswith("yaxis"):
        axis = getattr(fig.layout, attr)
        if axis:
            axis.showticklabels = True
        
fig

## Interpretation 

https://chatgpt.com/c/680a1854-fe68-800c-9642-8544ba7e471b

Use **additive** or **multiplicative** decomposition based on how the seasonal fluctuations behave relative to the trend:

---

### ✅ Use **additive** when:
- The **magnitude** of seasonal changes stays **constant** over time.
- The seasonal pattern does **not scale** with the trend.
- Example: sales increase over time, but holiday peaks remain around +20 units consistently.

### ✅ Use **multiplicative** when:
- The **magnitude** of seasonality **grows or shrinks** with the trend.
- The seasonal pattern **scales proportionally** to the level of the series.
- Example: if overall sales double, holiday peaks also double.

---

### Quick visual test:
- If your seasonal component has **roughly equal amplitude** throughout → **additive**.
- If your seasonal component’s **amplitude increases or decreases with the trend** → **multiplicative**.

---

Looking at your plot:
- The **seasonal amplitude** for the `Passengers` series seems to **increase** over time (especially right side).
- So, the **multiplicative model** is likely the better fit.

Let me know if you want to formally test this or visualize diagnostics.

Based on the decomposition plots and assuming this is airline passenger data (which it looks like), here are the **key insights** and **meaningful conclusions** you can derive:

---

### 📈 **1. Strong Upward Trend**
- Both additive and multiplicative models show a **clear long-term increase** in passengers over time.
- This suggests **growing demand** for air travel during the period (likely 1949–1960).

---

### 🔁 **2. Clear Seasonality**
- There is a **repeating seasonal pattern** every 12 months:
  - Peaks in mid-year (e.g., summer travel).
  - Troughs in early year (e.g., January).
- This regular pattern can be used for **forecasting** future seasonality.

---

### 📊 **3. Additive vs. Multiplicative**
- In the **additive model**, seasonal effects are **constant in magnitude**.
- In the **multiplicative model**, seasonal effects **grow with the trend**.
- Visually, the **multiplicative model better fits** the growing amplitude of the original data, especially toward the end of the series.

---

### 📉 **4. Residuals Show Model Fit**
- Residuals in the **multiplicative model are tighter** and more stable, especially in later years where the additive model leaves large unexplained spikes.
- This suggests the **multiplicative model better captures the structure** of the data.

---

### 🎯 **5. Model Result Approximates the Original**
- The final row (`model_result`) in the multiplicative model more closely follows the original `Passengers` series — especially where the amplitude is higher.

---

### 🧠 Strategic conclusions:
- **Forecasting models** (like Holt-Winters or SARIMA) should likely use a **multiplicative seasonal component** for this dataset.
- This time series is driven by **long-term growth** and **scaling seasonal effects**, not just fixed monthly deviations.
- **Residual diagnostics** (e.g., randomness, ACF) would help confirm this statistically.

---

Let me know if you want to formalize these conclusions into bullet points for a report or class presentation.